<a href="https://colab.research.google.com/github/abhay43/ML_Code/blob/master/RL_Block_Problem_AS12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The mouse can take actions in each direction  (UP=0, RIGHT=1, DOWN=2, 
LEFT=3). 

Model of the maze environment, wherein the states are defined is provided for 

your reference.  Any action that takes the mouse beyond the maze will result in 

the mouse staying in the same state.  Mouse receives a reward of -1 at each 

step until it reaches the cheese block.

Carry out the following tasks:

1.  Find an optimal  policy through Value  Iteration method that takes the 
         
       mouse to the cheese block starting from any internal state.

2.  Print the optimal policy to reach the cheese block.

3.   Print your output to a csv file.

 Output Format:
   
 Write the output  received towards the end in output.csv, which should be

 present at the location /code/output/output.csv

Example:  output.csv will have data that looks as given below: 

Given below is the sample output just for reference ﻿


In [0]:
import numpy as np
import sys
import pandas as pd

ACTION_UP    =  0
ACTION_RIGHT =  1
ACTION_DOWN  =  2
ACTION_LEFT  =  3

class GridWorld(object):
    def __init__(self,shape=[5,5]):
        self.shape = shape
        numStates  = shape[0] * shape[1]
        numActions = 4
        self.numStates = numStates
        self.numActions = numActions        
        xmax = shape[0]
        ymax = shape[1]

        grid = np.arange(numStates).reshape(shape)
        Model = {}

        x_indices = np.arange(xmax)
        y_indices = np.arange(ymax)

        for x in x_indices:
            for y in y_indices:
                state = y + x*(xmax)
                #print(x,y,state)
                Model[state] ={action:[] for action in np.arange(numActions)}

                is_terminal_state = lambda state : state == 0 or state == (numStates-1)
                reward = 0.0 if is_terminal_state(state) else -1.0


                if is_terminal_state(state):
                    Model[state][ACTION_UP] = [(1.0,state,reward,True)]
                    Model[state][ACTION_RIGHT] = [(1.0,state,reward,True)]
                    Model[state][ACTION_DOWN] = [(1.0,state,reward,True)]
                    Model[state][ACTION_LEFT] = [(1.0,state,reward,True)]
                else:
                    next_state = {}
                    next_state[ACTION_UP] = state if x == 0 else state - ymax
                    next_state[ACTION_RIGHT] = state if y == ymax-1 else state +1
                    next_state[ACTION_DOWN] = state if x == xmax-1 else state + ymax
                    next_state[ACTION_LEFT] = state if y == 0 else state -1 
                    Model[state][ACTION_UP] = [(1.0,next_state[ACTION_UP] ,reward,is_terminal_state(next_state[ACTION_UP]))]
                    Model[state][ACTION_RIGHT] = [(1.0,next_state[ACTION_RIGHT],reward,is_terminal_state(next_state[ACTION_RIGHT]))]
                    Model[state][ACTION_DOWN] = [(1.0,next_state[ACTION_DOWN],reward,is_terminal_state(next_state[ACTION_DOWN]))]
                    Model[state][ACTION_LEFT] = [(1.0,next_state[ACTION_LEFT],reward,is_terminal_state(next_state[ACTION_LEFT]))]
        self.model = Model

# Write a function to calculate the value for all the actions in a given state  

# def value_iteration(env, theta=0.0001, discount_factor=1.0):   
 #  print('test')

    # Write your code here
    
# Write the code to create a deterministic policy by using the optimal value function. Print the optimal policy.

     # Write your code here
    
# Save the final output as described in the sample format given below 

# data= np.array([[1, 2, 3, 4, 5], [0, 0, 4, 1, 2],[3, 4, 1, 0, 2],[2, 1, 3, 4, 5],[2, 1, 3, 4, 5]])
# output=pd.DataFrame(data)
# output.to_csv('/code/output/output.csv', header=False, index=False) 
        

In [0]:
def value_eveluation(policy, env, discount_factor=1.0, theta=0.00001):
  value_f = np.zeros(env.numStates)
  while True:
    delta = 0
    for state in range(env.numStates):
      state_value =0
      for action,action_prob in enumerate(policy[state]):
        for  prob, next_state, reward, done in env.model[state][action]:
          state_value += action_prob * prob * (reward + discount_factor * value_f[next_state])
        delta = max(delta, np.abs(state_value - value_f[state]))
        value_f[state] = state_value
        if delta < theta:
            break
    return value_f



In [0]:
env = GridWorld()
def value_iteration(env, policy_eval_fn=value_eveluation, discount_factor=1.0):
  
  def compute_value_fn_update(state,value_fn):
    value_fn_update = np.zeros(env.numActions)
    for action in range(env.numActions):
        for prob,next_state,reward,done in env.model[state][action]:
            value_fn_update[action] += prob * (reward + discount_factor * value_fn[next_state])                
    return value_fn_update 

  while True:    
    policy = np.ones([env.numStates,env.numActions]) /env.numActions
    value_f = value_eveluation(policy, env, discount_factor)
    policy_stable = True
    # improve the policy
    for state in range(env.numStates):
      select_action = np.argmax(policy[state])
      valueFun = compute_value_fn_update(state,value_f )
      best_action = np.argmax(valueFun)
      if select_action != best_action:
        policy_stable = False
      policy[state] = np.eye(env.numActions)[best_action]
      if policy_stable:
        return policy, value_f

  while True:
    delta = 0
    for state in range(env.numStates):
      state_value = 0
      for action,action_prob in enumerate(policy[state]):
              for  prob, next_state, reward, done in env.model[state][action]:
                  state_value += action_prob * prob * (reward + discount_factor * value_f[next_state])
              delta = max(delta, np.abs(state_value - value_f[state]))
              value_f[state] = state_value
      # Stop evaluating once our value function change is below a threshold
      if delta < theta:
          break
  return value_f


In [0]:
policy, v = value_iteration(env)

In [42]:
v.reshape(5,5)

array([[ 0.        , -1.        , -1.25      , -1.3125    , -1.390625  ],
       [-1.1875    , -1.546875  , -1.69921875, -1.75292969, -1.93530273],
       [-1.55859375, -1.77636719, -1.86889648, -1.90545654, -2.14364624],
       [-1.67456055, -1.86273193, -1.9329071 , -1.95959091, -2.22228718],
       [-1.99787521, -2.20657253, -2.2806766 , -2.30754131,  0.        ]])

In [0]:
output=pd.DataFrame(v.reshape(5,5))

In [44]:
output.head(100)

,0,1,2,3,4
0,0.000000,-1.000000,-1.250000,-1.312500,-1.390625
1,-1.187500,-1.546875,-1.699219,-1.752930,-1.935303
2,-1.558594,-1.776367,-1.868896,-1.905457,-2.143646
3,-1.674561,-1.862732,-1.932907,-1.959591,-2.222287
4,-1.997875,-2.206573,-2.280677,-2.307541,0.000000
